# 이상값 처리하기

## Iris 데이터 받기

In [1]:
import pandas as pd    # pandas package import

# 데이터프레임의 컬럼명 지정

col = ["꽃받침길이", "꽃받침너비", "꽃잎길이", "꽃잎너비", "붓꽃종류"]

# 웹에 있는 CSV 파일 읽어와서 데이터프레임 df로 넣는다.

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", names = col)

df

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비,붓꽃종류
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


- 이상값 강제 삽입

In [2]:
df.iloc[0,0] = 50
df.iloc[1,0] = 55
df.iloc[0,1] = 40
df.iloc[1,1] = 100


In [3]:
df  # 이상갑이 삽입된 데이터셋 확인하기

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비,붓꽃종류
0,50.0,40.0,1.4,0.2,Iris-setosa
1,55.0,100.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


## IQR Rule for Outliers (Interquartile Range) 기법

### 꽃받침길이에 관한 이상치 탐지

In [4]:
df1 = df["꽃받침길이"]   # 꽃받침길이 열만 떼어내어 df1에 저장하기
df1

0      50.0
1      55.0
2       4.7
3       4.6
4       5.0
       ... 
145     6.7
146     6.3
147     6.5
148     6.2
149     5.9
Name: 꽃받침길이, Length: 150, dtype: float64

In [5]:
# 4분위 수 정하기

q1 = df1.quantile(0.25)    # 1사분위 지점
q3 = df1.quantile(0.75)     # 3사분위 지점

print("1사분위지점: ", q1)
print("3사분위지점: ",q3)

1사분위지점:  5.1
3사분위지점:  6.4


In [6]:
# IQR 계산하기

iqr = q3 - q1
print(iqr)

1.3000000000000007


In [7]:
# 상한값, 하한값 계산

lower = q1 - 1.5*iqr
upper = q3 + 1.5*iqr

print("lower: ",lower)
print("upeer: ",upper)

lower:  3.1499999999999986
upeer:  8.350000000000001


In [8]:
# 상한값, 하한값을 벗어나는 데이터의 인덱스 값 계산

condition = (df1 > upper) | (df1 < lower)      # df1이 upper 보다 크거나, df1이 lower 보다 작은 데이터가 있는 경우 해당행에 True를 저장

In [9]:
outlier_index = df1.loc[condition].index.values   # 조건에 맞는 index 값을 가지고 옴
print("이상치가 있는 index: ",outlier_index)    # 이상값이 있는 index값을 출력

이상치가 있는 index:  [0 1]


### 꽃받침너비에 관한 이상치 탐지

In [10]:
df1 = df["꽃받침너비"]

q1 = df1.quantile(0.25)    # 1사분위 지점
q3 = df1.quantile(0.75)     # 3사분위 지점

print("1사분위지점: ", q1)
print("3사분위지점: ",q3)

iqr = q3 - q1
print(iqr)


lower = q1 - 1.5*iqr
upper = q3 + 1.5*iqr

print("lower: ",lower)
print("upeer: ",upper)

condition = (df1 > upper) | (df1 < lower)      # df1이 upper 보다 크거나, df1이 lower 보다 작은 데이터가 있는 경우 해당행에 True를 저장
outlier_index = df1.loc[condition].index.values   # 조건에 맞는 index 값을 가지고 옴
print("이상치가 있는 index: ",outlier_index)     # 이상값이 있는 index값을 출력


1사분위지점:  2.8
3사분위지점:  3.3
0.5
lower:  2.05
upeer:  4.05
이상치가 있는 index:  [ 0  1 15 32 33 60]


In [11]:
df1[outlier_index]    # 이상치가 있는 행 출력해 보기

0      40.0
1     100.0
15      4.4
32      4.1
33      4.2
60      2.0
Name: 꽃받침너비, dtype: float64

## 전체 열에 이상치 탐색하기

- df frame 전체 열을 대상으로 이상치를 탐색한다.

In [20]:
Q1 = df.select_dtypes(include='number').quantile(0.25)    # 1사분위 지점
Q3 = df.select_dtypes(include='number').quantile(0.75)    # 3사분위 지점

print(Q1)
print(Q3)

꽃받침길이    5.1
꽃받침너비    2.8
꽃잎길이     1.6
꽃잎너비     0.3
Name: 0.25, dtype: float64
꽃받침길이    6.4
꽃받침너비    3.3
꽃잎길이     5.1
꽃잎너비     1.8
Name: 0.75, dtype: float64


In [21]:
IQR = Q3 - Q1   # 전체 열에 대해서 수행이 된다.
print(IQR)

꽃받침길이    1.3
꽃받침너비    0.5
꽃잎길이     3.5
꽃잎너비     1.5
dtype: float64


In [22]:
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR

print("lower: ",lower)
print("upeer: ",upper)

lower:  꽃받침길이    3.15
꽃받침너비    2.05
꽃잎길이    -3.65
꽃잎너비    -1.95
dtype: float64
upeer:  꽃받침길이     8.35
꽃받침너비     4.05
꽃잎길이     10.35
꽃잎너비      4.05
dtype: float64


In [ ]:
numeric_df = df.select_dtypes(include='number')   # 수치형 컬럼만 따로 추출
condition = (numeric_df > upper) | (numeric_df < lower)      # df가 upper 보다 크거나, df가 lower 보다 작은 데이터가 있는 경우 해당행에 True를 저장
condition = condition.any(axis= 1)   # df 전체에서 열방향으로 조건값(df > upper) | (df < lower) 필터 수행
outlier_df = df[condition]      # 조건값(최댓값 최솟값에 벗어나는 데이터)들만 outlier_df에 저장하기
outlier_df


,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비,붓꽃종류
0,50.0,40.0,1.4,0.2,Iris-setosa
1,55.0,100.0,1.4,0.2,Iris-setosa
15,5.7,4.4,1.5,0.4,Iris-setosa
32,5.2,4.1,1.5,0.1,Iris-setosa
33,5.5,4.2,1.4,0.2,Iris-setosa
60,5.0,2.0,3.5,1.0,Iris-versicolor


In [29]:
outlier_index = df.loc[condition].index.values   # 조건에 맞는 index 값을 가지고 옴
outlier_index      # 이상치가 있는 index값

array([ 0,  1, 15, 32, 33, 60], dtype=int64)

### 이상값 처리

- 이상값에 대한 index를 구했으니, 이를 대체하거나 삭제하는 처리작업을 수행해야함

- 이상값 제거

In [30]:
df_new = df.drop(outlier_index)    # 이상값 index 행을 모두 제거

In [31]:
df_new

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비,붓꽃종류
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


#### 표준화 점수를 이용한 이상값 식별

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# 예제 데이터프레임 생성 (df_sample.csv과 유사)
data = {
    '학번': ['S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09', 'S10'],
    '중간': [90, 82, 80, 78, 93, 71, 60, 72, 65, 85],
    '기말': [95, 83, 78, 75, 91, 75, 80, 65, 65, 78],
    '리포트': [20, 18, 18, 10, 12, 16, 18, 14, 14, 10],
    '퀴즈': [20, 18, 18, 10, 12, 16, 18, 14, 14, 10]
}
df = pd.DataFrame(data)

# 중간 점수의 Z-Score 계산
df['중간_Z점수'] = zscore(df['중간'])

# Z-Score 기준으로 이상값 여부 판단 (예: |Z| > 2)
df['이상값_여부'] = df['중간_Z점수'].abs() > 2

# 결과 출력
print(df[['학번', '중간', '중간_Z점수', '이상값_여부']])

    학번  중간    중간_Z점수  이상값_여부
0  S01  90  1.231167   False
1  S02  82  0.436866   False
2  S03  80  0.238290   False
3  S04  78  0.039715   False
4  S05  93  1.529030   False
5  S06  71 -0.655299   False
6  S07  60 -1.747463   False
7  S08  72 -0.556011   False
8  S09  65 -1.251025   False
9  S10  85  0.734729   False
